In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler


In [2]:
df1=pd.read_csv('pkt8_data.csv')

C:\Users\arind\AppData\Local\Temp\ipykernel_11552\1635037348.py:1: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df1=pd.read_csv('pkt8_data.csv')


In [3]:
df1.head()

,No.,Time,Source,Destination,Protocol,Length,Info,RT
0,1,0.000000,1803.0,445.0,TCP,64,1803 > 445 [ACK] Seq=1 Ack=1 Win=65535 Len=0,CS0
1,2,0.000514,1687.0,443.0,UDP,139,1687 > 443 Len=50,CS0
2,3,0.000656,1803.0,445.0,TCP,64,1803 > 445 [ACK] Seq=1 Ack=1861 Win=65535 Len=0,CS0
3,4,0.000824,1687.0,443.0,UDP,203,1687 > 443 Len=50,CS0
4,5,0.000965,1803.0,445.0,NBSS,121,NBSS Continuation Message,CS0


In [4]:
df1.tail()

,No.,Time,Source,Destination,Protocol,Length,Info,RT
989502,989503,233.286529,3389.0,1904.0,SSL,96,Continuation Data,CS0
989503,989504,233.286698,NaN,NaN,LLC,68,"U, func=UI; SNAP, OUI 0x00000C (Cisco Systems,...",NaN
989504,989505,233.286895,80.0,36939.0,TCP,1518,80 > 36939 [ACK] Seq=1 Ack=308 Win=64240 Len...,CS0
989505,989506,233.286920,80.0,36939.0,TCP,1518,"80 > 36939 [PSH, ACK] Seq=1461 Ack=308 Win=6...",CS0
989506,989507,233.286941,80.0,36939.0,TCP,1518,"80 > 36939 [PSH, ACK] Seq=2921 Ack=308 Win=6...",CS0


In [6]:
df1.isna().sum()

No.                 0
Time                0
Source         143284
Destination    143284
Protocol            0
Length              0
Info             1572
RT              80041
dtype: int64

In [7]:
df1_cleaned=df1.dropna()
df1_cleaned.isna().sum()

No.            0
Time           0
Source         0
Destination    0
Protocol       0
Length         0
Info           0
RT             0
dtype: int64

In [8]:
len(df1_cleaned)

844643

In [16]:
df1_sampled_mili= df1_cleaned[df1_cleaned['No.']%4==0]

In [17]:
len(df1_sampled_mili)

211309

In [20]:
df2=pd.read_csv('pkt8_datarate.csv')

In [21]:
df2.head()

,Interval start,All Packets
0,0.000,591
1,0.001,7833
2,0.002,1419
3,0.003,158
4,0.004,139


In [22]:
df2.tail()

,Interval start,All Packets
233282,233.282,1790
233283,233.283,913
233284,233.284,334
233285,233.285,446
233286,233.286,6634


In [23]:
len(df2)

233287

In [24]:
df2.isna().sum()

Interval start    0
All Packets       0
dtype: int64

In [26]:
df_merged=pd.merge_asof(df1_sampled_mili,df2,left_on='Time',right_on='Interval start',direction='backward')
df_merged_1=df_merged[df_merged['Time']-df_merged['Interval start']<=0.0009]

In [28]:
df_merged_1.head()

,No.,Time,Source,Destination,Protocol,Length,Info,RT,Interval start,All Packets
0,4,0.000824,1687.0,443.0,UDP,203,1687 > 443 Len=50,CS0,0.000,591
1,8,0.001489,445.0,1803.0,NBSS,768,[TCP Previous segment not captured] NBSS Conti...,CS0,0.001,7833
3,20,0.006544,1803.0,445.0,TCP,70,[TCP Dup ACK 18#1] 1803 > 445 [ACK] Seq=64 A...,CS0,0.006,225
4,24,0.008812,1803.0,445.0,NBSS,121,NBSS Continuation Message,CS0,0.008,388
5,28,0.009382,445.0,1803.0,NBSS,768,NBSS Continuation Message,CS0,0.009,9124


In [30]:
df_merged_2=df_merged_1.drop(columns=['No.','Time','Info','RT','Interval start'])

In [32]:
df_merged_2.head()

,Source,Destination,Protocol,Length,All Packets
0,1687.0,443.0,UDP,203,591
1,445.0,1803.0,NBSS,768,7833
3,1803.0,445.0,TCP,70,225
4,1803.0,445.0,NBSS,121,388
5,445.0,1803.0,NBSS,768,9124


In [33]:
len(df_merged_2)

190216

In [34]:
df_merged_2.isna().sum()

Source         0
Destination    0
Protocol       0
Length         0
All Packets    0
dtype: int64

In [35]:
df_merged_2.to_csv('filtered_joined_packet_data.csv')

In [37]:
unique_values=df_merged_2['Protocol'].unique()
num_unique_values=df_merged_2['Protocol'].nunique()

print(str(unique_values) + ' ' +str(num_unique_values))

['UDP' 'NBSS' 'TCP' 'LLC' 'NBNS' 'DNS' 'SSL' 'Syslog' 'NTP' 'SSH' 'TELNET'
 'BOOTP' 'NBDS' 'Gryphon' 'DSI' 'UDPENCAP' 'ICMP' 'LLMNR' 'MDNS' 'RIP'
 'TCP/MIKEY' 'SMTP' 'PPTP' 'LPD' 'SAMETIME' 'FTP' 'DHCPv6'] 27


In [42]:
''' 
UDP - 17
NBSS - TCP
TCP - 6
LLC - UDP
NBNS - UDP
DNS - TCP
SSL - TCP
syslog - udp
ntp - udp
ssh - tcp
telnet - tcp
bootp - udp
nbds - udp
gryphon - tcp
dsi - tcp
udpencap - udp
icmp - 1
llmnr - tcp
mdns - tcp
rip - tcp
TCP/MIKEY - tcp
SMTP - tcp
PPTP - tcp
LPD - tcp
SAMETIME - tcp
FTP - tcp
DHCPv6 - udp
'''

tcp_list = ['NBSS','DNS','SSL','SSH','TELNET','Gryphon','DSI','LLMNR','MDNS','RIP','TCP/MIKEY','SMTP','PPTP','LPD','SAMETIME','FTP']
udp_list=['LLC','NBNS','Syslog','NTP','BOOTP','NBDS','UDPENCAP','DHCPv6']
def replace_protocol(value):
    if value in tcp_list:
        return "TCP"
    elif value in udp_list:
        return "UDP"
    else:
        return value
df_merged_3=df_merged_2
df_merged_3['Protocol']=df_merged_2['Protocol'].apply(replace_protocol)




In [43]:
df_merged_3.head()

,Source,Destination,Protocol,Length,All Packets
0,1687.0,443.0,UDP,203,591
1,445.0,1803.0,TCP,768,7833
3,1803.0,445.0,TCP,70,225
4,1803.0,445.0,TCP,121,388
5,445.0,1803.0,TCP,768,9124


In [44]:

unique_values=df_merged_3['Protocol'].unique()
num_unique_values=df_merged_3['Protocol'].nunique()

print(str(unique_values) + ' ' +str(num_unique_values))

['UDP' 'TCP' 'ICMP'] 3


In [45]:
df_merged_3.to_csv('filtered_joined_packet_data.csv')